### torch.optim
#### 训练过程中有一步是更新参数，不管是batch_size随机梯度下降法，还是带Adam或者动量的梯度下降法，都需要设定一些超参数，如学习率或者动量值等，再对模型中的超参数根据求得的导数和这些超参数进行更新。这个更新的过程可以使用torch.optim来完成。这个过程一般常见的代码如下：

In [ ]:
#省略前面的代码
optimizer = torch.optim.Adam(model.parameters(), 0.005)

loss_plot, c0_plot = [], []
# 迭代10个Epoch
for epoch in range(10):
    for data in train_loader:
        c0, c1, c2, c3, c4, c5 = model(data[0])
        loss = criterion(c0, data[1][:, 0]) + \
                criterion(c1, data[1][:, 1]) + \
                criterion(c2, data[1][:, 2]) + \
                criterion(c3, data[1][:, 3]) + \
                criterion(c4, data[1][:, 4]) + \
                criterion(c5, data[1][:, 5])
        loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_plot.append(loss.item())
        c0_plot.append((c0.argmax(1) == data[1][:, 0]).sum().item()*1.0 / c0.shape[0])
        
    print(epoch)

#### 其中涉及到更新参数的代码是如下几条：
optimizer = torch.optim.Adam(model.parameters(), 0.005)
##### 这行代码通过torch.optim.Adam类实例化出了一个优化器
optimizer.zero_grad()
##### 这行代码调用了生成的优化器的zero_grad()功能，让传入的model.parameters()中的所有参数的导数都归零
optimizer.step()
##### 这行代码调用了生成的优化器的step()功能，更新了model.parameters()中的所有参数
##### 这三行代码就是一个普通模型更新一次参数需要用到的所有代码。下面更深层次的来理解这三行代码：
像torch.optim.Adam()这样的类还有很多，例如torch.optim.SGD()、torch.optim.Adadelta()等，他们都有一个共同的基类：torch.optim.Optimizer(params, defaults)，而zero_grad()和step()方法都是这个基类的方法。这样来看就清晰多了。
##### 也就是说torch官方写了一个torch.optim.Optimizer基类，这个基类包含了很多参数优化需要的功能，同时官方也基于这个基类写了一系列可以直接调用的包含某些具体方法的子类，这样很多普遍使用的优化方法，像Adam，我们就可以不用自己去写，而是直接调用了。下面来看看torch.optim.Optimizer(params, defaults)基类的一些基本内容：
##### 1、参数
###### params(iterable) : 一个可以迭代的对象，或者是以字典的形式传入，内容是网络模型中需要优化的参数。这里的可迭代对象可以是生成器，正好跟torch.nn.parameters()方法返回的对象相吻合。
###### defaluts(dict) : 一系列键值对来明确一些优化器超参数，像学习率或者其他
##### 2、方法
###### add_param_group(param_group (dict) )：在已有参数中新增需要优化的参数
###### state_dict/load_state_dict : 获取或者载入优化器当前的所有参数和缓存
###### step ： 这个方法在类继承的时候需要重写，用于具体更新一次参数，在Adam和SGD这种官方写好的类中可以直接调用，无需重写，如果自己想要编写一个新的更新类的话，需要重写这个方法
###### zero_grad： 将优化器中所有参数的导数重置为0